# Atkinson & Boore (2003)

Troubleshooting

In [ ]:
%matplotlib inline
%load_ext autoreload

## Processing of coefficients

In [ ]:
import os
import glob
import warnings
import importlib
import pandas as pd
import numpy as np
import toolbox as tb
import matplotlib.pyplot as plt
from scipy.constants import g
from matplotlib.offsetbox import AnchoredText
from IPython.display import display
from openquake.hazardlib import imt

## Testing of GMPE

In [ ]:
%autoreload 2
import gmpe_tools as gt

from openquake.hazardlib import gsim

%autoreload 2
from openquake.hazardlib.gsim.atkinson_boore_2003 \
    import (
        AtkinsonBoore2003SSlabCascadiaNSHMP2008,
        AtkinsonBoore2003SSlabJapanNSHMP2008
    )

In [ ]:
gmpe_group = 'AB02'
gmpes = [
    AtkinsonBoore2003SSlabCascadiaNSHMP2008(),
    AtkinsonBoore2003SSlabJapanNSHMP2008(),
]
gmpe_class_names = [gmpe.__class__.__name__ for gmpe in gmpes]
gmpes_short = [
    'Cascadia',
    'Japan',
]
gmpes_long = [
    'Cascadia',
    'Japan',
]

In [ ]:
for gmpe in gmpes:
    gt.print_gmpe_summary(gmpe)

In [ ]:
means_files = ['']
stddev_files = gt.write_test_data(df_stddevs, gmpe_group, gmpes_short, gmpe_class_names, 'FIG4', float_format='%.5g')

In [ ]:
means_file

In [ ]:
df_ref = gt.df_massage(pd.read_csv(means_file))
print set(df_ref['rup_hypo_depth'])
df_ref.head()

In [ ]:
# run through unittests
threshhold = 0.1

gmpes = [
    AtkinsonBoore2003SSlabJapanNSHMP2008(),
    AtkinsonBoore2003SSlabCascadiaNSHMP2008(),
]

# note that I ended up going with a version which deleted amplitudes over 0.25 g; this version shows all amplitudes
means_files = [
    '/home/nick/Desktop/Individual Study/GMPEs/atkinson2003empirical/AB03SSlabJapanNSHMP_MEAN.csv',
    '/home/nick/src/python/GEM/oq-hazardlib/openquake/hazardlib/tests/gsim/data/AB03/AB03SSlabCascadiaNSHMP_MEAN.csv',
]
display(means_files)

fig, axes = plt.subplots(len(means_files), 1, figsize=(8, 6*len(means_files)))

for ax, means_file, gmpe in zip(axes, means_files, gmpes):
    gmpe_name = gmpe.__class__.__name__
    ax.add_artist(AnchoredText(gmpe_name, 
                               loc=tb.LOC_CODE['upper left'], frameon=False))
    ax.add_artist(AnchoredText(os.path.split(means_file)[1], 
                               loc=tb.LOC_CODE['upper right'], frameon=False))

    df_ref = gt.df_massage(pd.read_csv(means_file))
    #df_ref.drop(u'pga', axis=1, inplace=True)
    df_ref.drop([0.1, 0.2, 1.0, 2.0], axis=1, inplace=True)
    df_ref.rename(columns={'pga': 'PGA', 'pgv': 'PGV', 'pgd': 'PGD'}, inplace=True)

    mags = np.sort(np.array(list(set(df_ref['rup_mag']))))
    rakes = np.sort(np.array(list(set(df_ref['rup_hypo_depth']))))
    distances = sorted(list(set(df_ref['dist_rrup'])))
    vs30s = np.sort(np.array(list(set(df_ref['site_vs30']))))
    sa_cols = np.array([tb.is_numeric(item) for item in df_ref.columns])
    imt_cols = np.array([gt.is_imt(item) for item in df_ref.columns])
    im_types = [gt.get_imt(item) for item in df_ref.columns[imt_cols]]
    
    df_new = gt.compute_gmpe(gmpe, mags, rakes, distances, vs30s, im_types)[0]
    df_new = df_new[df_ref.columns]

    if sum(sa_cols) > 2:  # x-axis will be spectral acceleration period
        for vs30 in vs30s:
            for dist in distances:
                trace_label = '%g km, %g m/s' % (dist, vs30)

                df_trace = df_ref[
                    (df_ref['dist_rrup'] == dist) & 
                    (df_ref['site_vs30'] == vs30)]
                if df_trace.size == 0:
                    continue
                ref = df_trace[df_ref.columns[sa_cols]]

                df_trace = df_new[
                    (df_new['dist_rrup'] == dist) & 
                    (df_new['site_vs30'] == vs30)]
                new = df_trace[df_ref.columns[sa_cols]]

                h = ax.loglog(new.columns.values, new.transpose().values, 
                              label=trace_label, alpha=0.5)

                ax.loglog(ref.columns.values, ref.transpose().values, 
                          linestyle='none', marker='x', 
                          color=h[0].get_color(), alpha=0.5)
                
                # circle points in error
                error = (np.abs(new.transpose().values/new.transpose().values - 1) > threshhold).reshape((-1,))
                ax.loglog(np.repeat(new.columns.values, new.transpose().values.shape[1])[error],
                          new.transpose().values[error], 
                          linestyle='none', marker='o', color='black', markerfacecolor='none')
                
        ax.set_ylabel(str(im_type))
        ax.set_xlabel('Period [s]')
        
    else:  # x-axis will be distance (could be magnitude)
        for vs30 in vs30s:
            for mag in mags:
                for im_type in im_types:
                    trace_label = '%s, M=%g, %g m/s' % (str(im_type), mag, vs30)

                    df_trace_ref = df_ref[
                        (df_ref['rup_mag'] == mag) & 
                        (df_ref['site_vs30'] == vs30)]
                    if df_trace_ref.size == 0:
                        continue

                    h = ax.loglog(df_trace_ref['dist_rrup'], df_trace_ref[str(im_type)], 
                                  label=trace_label, alpha=0.5)

                    df_trace_new = df_new[
                        (df_new['rup_mag'] == mag) & 
                        (df_new['site_vs30'] == vs30)]

                    ax.loglog(df_trace_new['dist_rrup'], df_trace_new[str(im_type)], 
                              linestyle='none', marker='x', 
                              color=h[0].get_color(), alpha=0.5)
                    
                    # circle points in error
                    error = (np.abs(df_trace_new[str(im_type)]/df_trace_ref[str(im_type)] - 1) > threshhold).reshape((-1,))
                    ax.loglog(df_trace_new.loc[error, 'dist_rrup'], 
                              df_trace_new.loc[error, str(im_type)], 
                              linestyle='none', marker='o', color='black', markerfacecolor='none')
            
        ax.set_ylabel('Ground Motion')
        ax.set_xlabel('Distance [km]')
    
    ax.autoscale(enable=True, axis=u'x', tight=True)
    ax.legend(loc='center left', bbox_to_anchor=(1,0.5), frameon=False)

ax.add_artist(AnchoredText('points exeeding %g%% marked with circles' % (100*threshhold), 
                           loc=tb.LOC_CODE['lower left'], frameon=False))

plt.savefig('Unit_Test_Summary.pdf', dpi=300, bbox_inches='tight')
plt.savefig('Unit_Test_Summary.png', dpi=300, bbox_inches='tight')

In [ ]:
df_ref.sort(columns='PGA', ascending=False).head()

In [ ]:
df_new.sort(columns='PGA', ascending=False).head()